In [2]:
from __future__ import print_function
import concurrent.futures

import requests
from bs4 import BeautifulSoup
from collections import deque
import pandas as pd
import numpy as np
import time
import logging

from threading import Thread
from queue import Queue

from threading import Event

import os, shutil

from tqdm import tqdm

import urllib3
urllib3.disable_warnings()

import requests
requests.packages.urllib3.disable_warnings()

import warnings
warnings.filterwarnings('ignore')

In [4]:
LIMIT = 20
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
FOLDER = "C:/Matko - CloudStation/UPJS/_Bakalarka/data_drojove_kody_stranok_nove/"

In [3]:
df = pd.read_csv('dataset_unreachable_urls_filtered_with_lang_labels.csv', )
df = df.set_index('index')
df = df[df['lang_eng'] == 1] # Pracujeme iba s anglickym jazykom
print(df.shape)
df.head()
df.tail()

(50769, 4)


,name,link,type,lang_eng
index,,,,
101248,Erasmus University Rotterdam,http://www.eur.nl/english/,Education,1
101249,LifeStream,https://www.lstream.org/,Nonprofit,1
101252,Shirak State University,http://shsu.am/en/,Education,1
101255,Radboud University Nijmegen,http://www.ru.nl/english/,Education,1
101257,Northamptonshire Healthcare NHS Foundation Trust,https://www.nhft.nhs.uk/,Healthcare,1


In [4]:
# url_adresy = [[i, link] for i, link in enumerate(df['link']) if df.iloc[i, 4] == 1]
# url_adresy = [[i, link] for i, link in zip(df["index"], df["link"]) if df.loc[df["index"] == i]["lang_eng"].values[0] == 1]
# len(url_adresy)

In [5]:
def get_data_from_subpage(url):
    try:
        r = requests.get(url, headers=HEADERS, timeout=3, verify=False)
        return r.text
    except Exception as e:
        print(e)
        return None

def get_links_of_subpages(website_link):
    newly_found_links = deque()
    html_data = get_data_from_subpage(website_link)
    if html_data is None:
        return None, None
    try:
        soup = BeautifulSoup(html_data, "html.parser")
        for link in soup.find_all("a", href=True)[:LIMIT]:
            # print(link)

            # Append to list if new link contains original link
            if str(link["href"]).startswith((str(website_link))):
                newly_found_links.append(link["href"])

            # Include all href that do not start with website link but with "/"
            if str(link["href"]).startswith("/"):
                link_with_www = website_link + link["href"][1:]
                newly_found_links.append(link_with_www)

            # include subdomains i.e domain: upjs.sk, subdomain: science.upjs.sk
            if website_link.split("//")[1].split("/")[0].replace("www.", "") in str(link["href"]):
                newly_found_links.append(str(link["href"]))
        return newly_found_links, html_data
    except Exception as e:
        print(e)
        return None, None

def get_subpages_of_website(website_url, i):
    written_pages_iter = 0
    links_to_be_checked = deque([website_url])
    already_checked_links = deque()
    # file_name = f"../tmp//{i}.txt"

    while links_to_be_checked:
        link = links_to_be_checked.popleft()
        if link not in already_checked_links:
            new_links_of_subpages, page_content = get_links_of_subpages(link)
            if not (new_links_of_subpages is None and page_content is None):
                links_to_be_checked.extend(new_links_of_subpages)
                with open(f"{FOLDER}{i}/{written_pages_iter}.txt", "w", encoding='utf-8') as file:
                    file.write(link+"\n")
                    file.write(str(page_content))
                    written_pages_iter += 1

                # Print some statements
                total = len(links_to_be_checked) + len(already_checked_links)
                # print(f"Page_idx: {i}")
                # print(f"Progress: {written_pages_iter} / {LIMIT}\n")
                # print(f"Checked: {len(already_checked_links)} / {total}\n")
                # print(f"Written: = {written_pages_iter}")

        already_checked_links.append(link)

        # print(len(links_to_be_checked))
        # print(links_to_be_checked)

        if written_pages_iter >= LIMIT:
            break

    return True


def crawl(i, adresa):

    folder_name = FOLDER + str(i)

    try:
        page = requests.get(adresa, headers=HEADERS, timeout=5)
        if page.status_code == 200:

            if not os.path.exists(folder_name):
                os.mkdir(folder_name)

            get_subpages_of_website(adresa, i)

    except requests.exceptions.RequestException as e:
        logging.info(f'{i}: {adresa}: is Not reachable')


In [6]:
# Zistenie kolko podstranok mam stiahnutych ku ktorej url adrese

df = df.reindex(columns=df.columns.tolist() + ['ma_20_suborov'])

# Fill all values in column with 0 as default
df['ma_20_suborov'] = 0

for filename in os.listdir(FOLDER):
    pocet_suborov = len(os.listdir(FOLDER + str(filename)))
    if pocet_suborov == 20:
        df.at[int(filename), 'ma_20_suborov'] = 1   # If there are 20 files in directory

df.head()

,name,link,type,lang_eng,ma_20_suborov
index,,,,,
0,Australian National University,http://www.anu.edu.au/,Education,1,1
1,Monash University,http://www.monash.edu/,Education,1,1
2,University of Queensland,http://www.uq.edu.au/,Education,1,1
3,Macquarie University,http://mq.edu.au/,Education,1,1
4,UNSW Sydney,https://www.unsw.edu.au/,Education,1,1


In [7]:
# Stranky (priecinky), ktorym prislucha menej ako 20 suborov

df_recheck = df[df['ma_20_suborov'] == 0]

# Novy stlpec na trackovanie, ci je url adresa skontrolovana
df_recheck = df_recheck.reindex(columns=df_recheck.columns.tolist() + ['checked'])
df_recheck['checked'] = 0

print(df_recheck.shape)
df_recheck.head()

(21469, 6)


,name,link,type,lang_eng,ma_20_suborov,checked
index,,,,,,
8,University of Tasmania,http://www.utas.edu.au/,Education,1,0,0
24,University of Canberra,http://www.canberra.edu.au/,Education,1,0,0
31,QIMR Berghofer Medical Research Institute,http://www.qimrberghofer.edu.au/,Facility,1,0,0
34,Victor Chang Cardiac Research Institute,http://www.victorchang.edu.au/,Nonprofit,1,0,0
36,Mater Research,http://research.mater.org.au/,Facility,1,0,0


In [8]:
print(df_recheck.shape)
df_recheck = df_recheck.drop(labels=16236, axis=0)
print(df_recheck.shape)

(21469, 6)
(21468, 6)


In [ ]:
# Samotne stahovanie udajov zo stranok

od_idx, po_idx = 0, 10000
n_jobs = po_idx - od_idx

indexes = np.array(list(df_recheck.index.values))[od_idx:po_idx]
urls = np.array(list(df_recheck['link']))[od_idx:po_idx]

start = time.perf_counter() # calculate the time

with tqdm(total=(n_jobs)) as pbar:
    with concurrent.futures.ThreadPoolExecutor(max_workers=500) as executor: # In this case thread would be better
        # futures = {executor.submit(crawl, i, url): (i, url) for i, url in np.array(df)[od_idx:po_idx]} # cely dataset
        futures = {executor.submit(crawl, i, url): (i, url) for i, url in zip(indexes, urls)} # len stranky, ktore opakovane stahujeme
        results = {}
        for future in concurrent.futures.as_completed(futures):
            arg = futures[future]
            results[arg] = future.result()
            pbar.update(1)

end = time.perf_counter() # calculate finish time
print(f'Threads: Finished in {round(end - start,2)} second(s)')

  0%|          | 0/10000 [00:00<?, ?it/s]WARNING:urllib3.connection:Certificate did not match expected hostname: www.uofk.edu. Certificate: {'subject': ((('commonName', 'uofk.edu'),),), 'issuer': ((('countryName', 'US'),), (('organizationName', "Let's Encrypt"),), (('commonName', 'R3'),)), 'version': 3, 'serialNumber': '041EA742E2B013D1590C8B5B2C49C9AFDBCA', 'notBefore': 'Feb 15 06:15:13 2023 GMT', 'notAfter': 'May 16 06:15:12 2023 GMT', 'subjectAltName': (('DNS', 'uofk.edu'),), 'OCSP': ('http://r3.o.lencr.org',), 'caIssuers': ('http://r3.i.lencr.org/',)}
Traceback (most recent call last):
  File "C:\Users\Martin\DataspellProjects\bakalarka\venv\Lib\site-packages\urllib3\connectionpool.py", line 469, in _make_request
    assert_header_parsing(httplib_response.msg)
  File "C:\Users\Martin\DataspellProjects\bakalarka\venv\Lib\site-packages\urllib3\util\response.py", line 91, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.except

In [10]:
# Nepouzity kod multiprocesingu

# # Vlozenie jednotlivych taskov (stranok) do radu (Queue)
# threads = []
# q = Queue(maxsize=0)
# # n_jobs = min(1000000000, len(url_adresy))
# from_n_jobs, n_jobs = 0, 10000
#
#
# # results = [{} for x in url_adresy]
# results = [False for x in url_adresy]
# for i, url in url_adresy[from_n_jobs: n_jobs]:
#     q.put((i, url))
#
#
# # Vykonanie taskov pomocou multithreading-u
# cas = time.time()
#
# #Starting worker threads on queue processing
# for i in range(n_jobs):
#     logging.debug('Starting thread ', i)
#     worker = Thread(target=crawl, args=(q, results))
#     # worker.setDaemon(True)    #setting threads as "daemon" allows main program to exit eventually even if these dont finish correctly.
#     worker.daemon = True
#     worker.start()
#
#
# #now we wait until the queue has been processed
# q.join()
#
# # logging.info('All tasks completed.')
# print('All tasks completed.')
#
# print(f"Websites checked: {n_jobs}")
# print(f"Finished in {(time.time() - cas):.1f} seconds.")

In [11]:
# Odstranenie vsetkych suborov z lubovolneho priecinku definovaneho ako "folder"

folder = '../tmp'  # Change me

for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [5]:
# Statistika - kolko suborov je v jednotlivych priecinkoch

d = {}

websites_with_20_subpages_downloaded = []

print(f"Folder count: {len(os.listdir(FOLDER))}")
for filename in os.listdir(FOLDER):
    pocet_suborov = len(os.listdir(f'{FOLDER}{filename}'))
    if pocet_suborov == 20:
        websites_with_20_subpages_downloaded.append(int(filename))
    # print(f"{filename}: {pocet_suborov}")
    d[pocet_suborov] = d[pocet_suborov] + 1 if pocet_suborov in d else 1

print(d)
print(len(websites_with_20_subpages_downloaded))

Folder count: 46430
{20: 29322, 1: 5989, 17: 1178, 19: 1161, 0: 152, 18: 1238, 2: 785, 5: 169, 9: 276, 16: 1055, 13: 650, 8: 247, 3: 354, 4: 223, 15: 976, 6: 203, 11: 501, 10: 405, 14: 739, 7: 253, 12: 554}
29322
